In [53]:
import pandas as pd
import numpy as np

data = pd.read_parquet('train_data.parquet', engine='pyarrow')

In [46]:
data

,id,target,sample_ml_new,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,0,0,1,1761,1759,85,105469,191,46,0,...,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1,0,1,1761,1759,120,105610,144,71,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,1,890,1759,141,105227,191,11,0,...,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
5,5,0,1,1599,966,30,102441,191,8,0,...,0,6872,8530,0,0,0,0,0,6872,8530
6,6,0,1,1761,1759,85,104006,191,2,0,...,0,0,7642,0,0,0,0,0,0,7642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693036,693036,0,1,1761,1759,141,410,191,2,125,...,0,0,0,0,0,0,0,0,0,0
693038,693038,0,1,1761,1759,0,170,191,1,125,...,0,0,0,0,0,51714,0,0,0,0
693039,693039,0,1,1761,1759,0,168,191,1,125,...,0,0,0,0,0,51714,0,0,0,0
693040,693040,0,1,1761,1759,141,74,191,1,125,...,0,0,0,0,0,51714,0,0,0,0


In [96]:
data_without_nan = data.fillna(0)
data_without_nan.head()

,id,target,sample_ml_new,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,0,0,1,1761,1759,85,105469,191,46,0,...,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1,0,1,1761,1759,120,105610,144,71,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,1,890,1759,141,105227,191,11,0,...,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
5,5,0,1,1599,966,30,102441,191,8,0,...,0,6872,8530,0,0,0,0,0,6872,8530
6,6,0,1,1761,1759,85,104006,191,2,0,...,0,0,7642,0,0,0,0,0,0,7642


In [92]:
print(data[data.isna().any(axis=1)], sep="\n")


# index 0 feature756 nan
# index 315324 feature643, feature756 nan
# index 519615 feature756 nan

            id  target  sample_ml_new  feature1  feature2  feature3  feature4  \
0            0       0              1      1761      1759        85    105469   
1            1       0              1      1761      1759       120    105610   
2            2       0              1       890      1759       141    105227   
5            5       0              1      1599       966        30    102441   
6            6       0              1      1761      1759        85    104006   
...        ...     ...            ...       ...       ...       ...       ...   
693036  693036       0              1      1761      1759       141       410   
693038  693038       0              1      1761      1759         0       170   
693039  693039       0              1      1761      1759         0       168   
693040  693040       0              1      1761      1759       141        74   
693042  693042       1              1      1761      1759       141        28   

        feature5  feature6 

In [22]:
pd.isnull(data).sum().value_counts()

0         1077
315324       1
519615       1
Name: count, dtype: int64

In [12]:
print(*list(data.columns), sep="\n")

id
target
sample_ml_new
feature1
feature2
feature3
feature4
feature5
feature6
feature7
feature8
feature9
feature10
feature11
feature12
feature13
feature14
feature15
feature16
feature17
feature18
feature19
feature20
feature21
feature22
feature23
feature24
feature25
feature26
feature27
feature28
feature29
feature30
feature31
feature32
feature33
feature34
feature35
feature36
feature37
feature38
feature39
feature40
feature41
feature42
feature43
feature44
feature45
feature46
feature47
feature48
feature49
feature50
feature51
feature52
feature53
feature54
feature55
feature56
feature57
feature58
feature59
feature60
feature61
feature62
feature63
feature64
feature65
feature66
feature67
feature68
feature69
feature70
feature71
feature72
feature73
feature74
feature75
feature76
feature77
feature78
feature79
feature80
feature81
feature82
feature83
feature84
feature85
feature86
feature87
feature88
feature89
feature90
feature91
feature92
feature93
feature94
feature95
feature96
feature97
feature98
featu

In [20]:
data['sample_ml_new'].value_counts()

sample_ml_new
1    519615
Name: count, dtype: int64